In [1]:
import datetime as dt

Cosa sappiamo di river... abbiamo prima di tutto che le features sono rappresentate da dizionari in river.
Quindi abbiamo che la chiave corrisponde al nome della features. In questo caso SALES

In [3]:

x = {
    'shop': 'Ikea',
    'city': 'Stockholm',
    'date': dt.datetime(2020, 6, 1),
    'sales': 42
}

Possiamo decidere noi come far attivare la fase di stream, sicuramente River offre il modulo Stream che ha molte funzionalità utili per questo scopo... per esempio ```stream.iter_csv```.

Poi River comunque offre una serie di dataset che possono essere utilizzati per provare. ```dataset```

In [10]:
from river import datasets

dataset = datasets.Phishing()
print(dataset)

Phishing websites.

This dataset contains features from web pages that are classified as phishing or not.

    Name  Phishing                                                                                      
    Task  Binary classification                                                                         
 Samples  1,250                                                                                         
Features  9                                                                                             
  Sparse  False                                                                                         
    Path  /home/giodefa/miniconda3/envs/river/lib/python3.9/site-packages/river/datasets/phishing.csv.gz


In [16]:
for x, y in dataset:
    pass
print(x)

{'empty_server_form_handler': 1.0, 'popup_window': 0.5, 'https': 1.0, 'request_from_other_domain': 1.0, 'anchor_from_other_domain': 1.0, 'is_popular': 0.5, 'long_url': 0.0, 'age_of_domain': 0, 'ip_in_url': 0}


Possiamo vedere come abbiamo tutte le features utilizzate nel nostro caso! quindi abbiamo 9 features, normalizzate! E infine abbiamo la nostra label, quindi può essere definito un problema di classificazione!

In [17]:
print(y)

False


In [21]:
from river import linear_model

model = linear_model.LogisticRegression()

for x,y in dataset:
   model.learn_one(x, y)
    

In [24]:
model = linear_model.LogisticRegression()

for x,y in dataset:
    y_pred = model.predict_proba_one(x)
    y_pred_one = model.predict_one(x)
    print('pred_prb :', y_pred, 'pred_one: ', y_pred_one)
    model.learn_one(x, y)



pred_prb : {False: 0.5, True: 0.5} pred_one:  False
pred_prb : {False: 0.4971875296627105, True: 0.5028124703372895} pred_one:  True
pred_prb : {False: 0.49376965822249996, True: 0.5062303417775} pred_one:  True
pred_prb : {False: 0.49192870317923, True: 0.50807129682077} pred_one:  True
pred_prb : {False: 0.4854329829035461, True: 0.5145670170964539} pred_one:  True
pred_prb : {False: 0.488761369128878, True: 0.511238630871122} pred_one:  True
pred_prb : {False: 0.4958461672260852, True: 0.5041538327739148} pred_one:  True
pred_prb : {False: 0.49273510619655947, True: 0.5072648938034405} pred_one:  True
pred_prb : {False: 0.49304702630217423, True: 0.5069529736978258} pred_one:  True
pred_prb : {False: 0.4968133893502038, True: 0.5031866106497962} pred_one:  True
pred_prb : {False: 0.4918142273117062, True: 0.5081857726882938} pred_one:  True
pred_prb : {False: 0.49732316630931683, True: 0.5026768336906832} pred_one:  True
pred_prb : {False: 0.49307780403643875, True: 0.50692219596356

In [45]:
from river import metrics

model = linear_model.LogisticRegression()

metric = metrics.ROCAUC()

for x,y in dataset:
    y_pred = model.predict_proba_one(x)
    model.learn_one(x, y)
    metric.update(y,y_pred)

metric



ROCAUC: 89.36%

In [50]:
from river import compose
from river import preprocessing

model = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LogisticRegression()
)
model

Pipeline (
  StandardScaler (
    with_std=True
  ),
  LogisticRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.01
      )
    )
    loss=Log (
      weight_pos=1.
      weight_neg=1.
    )
    l2=0.
    intercept_init=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
)

In [66]:
metric = metrics.ROCAUC()

for x,y in datasets.Phishing():
    y_pred = model.predict_proba_one(x)
    model.learn_one(x,y)
    metric.update(y, y_pred)

metric

ROCAUC: 95.62%